In [50]:
# Check these before running
# KPUM Workstation
sourcedatabasefolder = '/mnt/d/Finn/KPUM_NODDI/DICOM_KPUM_NODDI/sourcedata'
studydir = ''
# Finn's Laptop
sourcedatabasefolder = '/mnt/d/Finn/KPUM_NODDI/DICOM_KPUM_NODDI/sourcedata'
studydir = '/mnt/s/Research/Projects/KPUM_NODDI/Data'

# nifti base folder to go within studydir
outputniftibasefolder = 'nifti'

# tsv-files to keep track of in studydir/outputniftibasefolder
subjecttrackerpath = os.path.join(studydir, outputniftibasefolder)
subjecttrackerfile = 'Subject_Tracker_for_sourcedata2nifti.tsv'

###################################################################################
# User input

# Participant details
subject = '0XX'
session = 'MR1'

# Define good DCM datafolder as input (and if no data is not good enough or does not exist, leave blank)

## ANATOMY
# High-resolution T1w MPRAGE
t1w_mprage = ''
# FLAIR
flair = ''
# T2star
t2star = ''
# T2w Cor
t2wtra = ''
# T2w Cor
t2wcor = ''
# SWI (if available)
swi = ''
# SWI minIP (if available)
swiminip = ''
# High-resolution T2w MCRIB (if available)
t2w_mcrib = 's05_T2W_MCRIB'

## DIFFUSION
# DKI dir-AP data
dki_dirAP = 's17_DKI_2.5mm_NODDI_dir-AP_79dir_0_1000_2000_DFC_MIX'
# DKI dir-PA data (if available)
dki_dirPA = 's16_DKI_2.5mm_NODDI_dir-PA_2dir_0_DFC_MIX'


###################################################################################

# Define local functions
def dicom2nifti(inputdicomdir, outputdicomdir):
    # Re-organize using dcm2niix
    import subprocess
    from datetime import datetime
    
    print("Start re-organising DCMs")    
    starttime=datetime.now()
    filebase = os.path.join("s%2s_%p","%p_%5r.dcm")
    p=subprocess.Popen(f"dcm2niix -a y -d 1 -b y -w 1 -o {outputdicomdir} -f {filebase} {inputdicomdir}", stdout=subprocess.PIPE, shell=True)
    (output, err) = p.communicate()  
    #This makes the wait possible
    p_status = p.wait()
    #This will give you the output of the command being executed
    #print(f"Command output: {output}")
    endtime=datetime.now()
    print("Done re-organising DCMs. Elapsed time:", endtime-starttime)


# Create conversion file sourcedata2nifti.tsv
import os, subprocess
import pandas as pd

# Define I/O folders
sourcedatainputfolder = os.path.join(sourcedatabasefolder, os.path.join(f'sub-{subject}',f'ses-{session}'))
niftioutputfolder = os.path.join(studydir, outputniftibasefolder, os.path.join(f'sub-{subject}',f'ses-{session}'))
if not os.path.exists(niftioutputfolder): # then make this directory
    os.makedirs(niftioutputfolder)

# Create list for matching file transfers
sourcedatalist = [t1w_mprage, 
                  t2w_mcrib,
                  t2wtra,
                  t2wcor,
                  flair,
                  t2star,
                  swi,
                  swiminip,
                  dki_dirAP,
                  dki_dirPA ] 

niftidatalist = [os.path.join('anat',f'sub-{subject}_ses-{session}_acq-mprage_T1w.nii'), 
                 os.path.join('anat',f'sub-{subject}_ses-{session}_acq-mcrib_T2w.nii'),
                 os.path.join('anat',f'sub-{subject}_ses-{session}_acq-tra_T2w.nii'),
                 os.path.join('anat',f'sub-{subject}_ses-{session}_acq-cor_T2w.nii'),
                 os.path.join('anat',f'sub-{subject}_ses-{session}_FLAIR.nii'),
                 os.path.join('anat',f'sub-{subject}_ses-{session}_acq-t2star_T2starw.nii'),
                 os.path.join('anat',f'sub-{subject}_ses-{session}_acq-swi_T2starw.nii'),
                 os.path.join('anat',f'sub-{subject}_ses-{session}_acq-swi_rec-mnip_T2starw.nii'),
                 os.path.join('dwi',f'sub-{subject}_ses-{session}_dir-AP_dwi.nii'),
                 os.path.join('fmap',f'sub-{subject}_ses-{session}_dir-PA_epi.nii'),
                 ]

# Make this into a DataFrame
sample = {'input': sourcedatalist, 
          'output': niftidatalist,
          'converted': ''
          'comments': ''}

df = pd.DataFrame(sample)
# Locate only the files that are going to be transferred (i.e. not have entry '' in sourcedatalist)
df_filtered = (df.loc[df['sourcedata_input'] != ''])
# and save to tsv-file in niftioutputfolder
df_filtered.to_csv(os.path.join(niftioutputfolder, f'sub-{subject}_ses-{session}_sourcedata2nifti.tsv'), sep="\t", index=False)

# Now use this df_filtered / tsv-file for the data transfer
# and perform conversion using dcm2niix
for ind in df_filtered.index:
        in_path = os.path.join(sourcedatainputfolder, df_filtered['sourcedata_input'][ind])
        out_filebase = os.path.splitext(df_filtered['nifti_output'][ind])[0] # workaround to get rid of file-extension
        print('Start converting DCMs in', in_path,'and save in', os.path.join(niftioutputfolder, out_filebase))
        p=subprocess.Popen(f"dcm2niix -a y -d 1 -b y -w 1 -o {niftioutputfolder} -f {out_filebase} {in_path}", stdout=subprocess.PIPE, shell=True)  
        print('Finished converting DCMs in', in_path,'and save in', os.path.join(niftioutputfolder, out_filebase))
        

